In [ ]:
!pip install hopsworks
#!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
#!sudo update-alternatives --config python3
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user
!which -a pip
!conda config --set offline false
!jupyter kernelspec list
sys.path.append('/usr/local/lib/python3.8/site-packages/')
import hopsworks

In [1]:
import pandas as pd
from datetime import datetime
!python -V
import sys;
print(sys.version)

Python 3.9.5
3.9.5 (default, Jun  4 2021, 12:28:51) 
[GCC 7.5.0]


In [2]:
# Initialization: Load shared functions and simulated data 
import os 

# Load shared functions
!curl -O https://raw.githubusercontent.com/Fraud-Detection-Handbook/fraud-detection-handbook/main/Chapter_References/shared_functions.py
%run shared_functions.py

# Get simulated data from Github repository
if not os.path.exists("simulated-data-transformed"):
    !git clone https://github.com/Fraud-Detection-Handbook/simulated-data-transformed

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63257  100 63257    0     0  73214      0 --:--:-- --:--:-- --:--:-- 73129


In [10]:
DIR_INPUT='simulated-data-transformed/data/' 
DIR_INPUT='simulated-data-raw/SEPA/'  # SEPA TRX

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"
BEGIN_DATE = "2023-01-14" # SEPA TRX
END_DATE = "2023-04-19" # SEPA TRX

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files


ValueError: No objects to concatenate

941940 transactions loaded, containing 295352 fraudulent transactions


In [11]:
print(transactions_df.TX_DATETIME.head(1))
print(transactions_df.TX_DATETIME.tail(1))

0   2023-01-14 00:00:51
Name: TX_DATETIME, dtype: datetime64[ns]
941939   2023-04-19 23:59:50
Name: TX_DATETIME, dtype: datetime64[ns]


In [12]:
# Setting the starting day for the training period, and the deltas
start_date_training = datetime.datetime.strptime(BEGIN_DATE, "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7

print(transactions_df.TX_AMOUNT.value_counts())

(train_df, test_df)=get_train_test_set(transactions_df,start_date_training,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)
print(train_df.TX_AMOUNT.value_counts())
print( train_df.shape)
print( test_df.shape)
# By default, scaling the input data
(train_df, test_df)=scaleData(train_df,test_df,input_features)

print(train_df.TX_AMOUNT.value_counts())

116.29     13
143.44     13
18.39      12
380.68     12
178.42     12
           ..
2625.77     1
4216.99     1
8261.05     1
1198.96     1
3316.19     1
Name: TX_AMOUNT, Length: 527368, dtype: int64
501.58     4
128.86     4
1437.70    4
1144.00    4
338.21     4
          ..
1723.91    1
1273.64    1
7058.96    1
4571.51    1
1760.62    1
Name: TX_AMOUNT, Length: 65074, dtype: int64
(68837, 37)
(18296, 37)
-0.836822    4
-0.957700    4
-0.533224    4
-0.628475    4
-0.889805    4
            ..
-0.440402    1
-0.586431    1
 1.289835    1
 0.483118    1
-0.428496    1
Name: TX_AMOUNT, Length: 65074, dtype: int64


In [13]:
[x.lower() for x in train_df.columns.to_list()]

['transaction_id',
 'tx_datetime',
 'customer_id',
 'iban',
 'bic',
 'fullname',
 'country',
 'ext_customer_id',
 'ext_iban',
 'ext_bic',
 'ext_name',
 'ext_country',
 'tx_operation_type',
 'tx_direction',
 'tx_status',
 'tx_sepa_scheme',
 'tx_code_status',
 'terminal_id',
 'tx_amount',
 'tx_time_seconds',
 'tx_time_days',
 'tx_fraud',
 'tx_fraud_scenario',
 'tx_during_weekend',
 'tx_during_night',
 'customer_id_nb_tx_1day_window',
 'customer_id_avg_amount_1day_window',
 'customer_id_nb_tx_7day_window',
 'customer_id_avg_amount_7day_window',
 'customer_id_nb_tx_30day_window',
 'customer_id_avg_amount_30day_window',
 'terminal_id_nb_tx_1day_window',
 'terminal_id_risk_1day_window',
 'terminal_id_nb_tx_7day_window',
 'terminal_id_risk_7day_window',
 'terminal_id_nb_tx_30day_window',
 'terminal_id_risk_30day_window']

In [14]:
test_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,IBAN,BIC,FULLNAME,COUNTRY,EXT_CUSTOMER_ID,EXT_IBAN,EXT_BIC,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
138189,833399,2023-01-28 00:01:37,1545,FR7614518000000001234567826,FTNOFRP1XXX,Keene Dody,FR,4522,FR7615589000000001234567858,CMBRFR2B,...,-2.114811,-0.850753,-2.398511,-1.203897,-0.153361,0.913000,0.826736,0.463893,-0.384058,-2.819765
138190,833400,2023-01-28 00:01:49,2166,Unknown country-code 'AM',Unknown country-code 'AM',Toss McClenaghan,AM,1806,FR7617499000000001234567812,BCADNCNNXXX,...,-1.721695,-0.592190,-2.398511,-0.691805,-0.153361,0.913000,0.822710,0.451210,-0.384058,-2.819765
138191,833401,2023-01-28 00:02:11,4650,FR7614707000000001234567883,CCBPFRPPXXX,Hibbert Jaqitsch,FR,4664,FR7612906000000001234567831,AGRIFRPP829,...,-0.149231,-0.760338,1.127832,-0.772364,-0.162603,0.919238,0.822710,0.451210,-0.392004,-2.815314
138199,833409,2023-01-28 00:07:01,3503,FR7644149000000001234567806,ESCBFRPP,Kemal Patchor,FR,596,Bank code exceeds maximum size 2,Bank code exceeds maximum size 2,...,-1.721695,0.312066,-1.979539,0.627090,-0.153361,0.893291,0.814658,0.388960,-0.415841,-2.872905
138213,833423,2023-01-28 00:17:39,2043,Invalid BBAN structure: '020041000000000000012...,Invalid BBAN structure: '020041000000000000012...,Weksler Winterberg,SC,1854,FR7610188000000001234567815,BCHAFR21,...,-0.280270,-1.426296,-1.211425,-1.576463,-0.171845,0.905763,0.846865,0.379782,-0.447624,-2.883480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206745,901955,2023-02-03 23:41:27,3712,Unknown country-code 'GU',Unknown country-code 'GU',Britni Anet,GU,1757,FR7630004000000001234567845,BNPAFRPPIFO,...,0.768039,-1.221102,0.604117,-1.252915,0.780086,-0.568837,2.888012,4.364923,7.220034,-3.444787
206751,901961,2023-02-03 23:46:48,642,Unknown country-code 'BB',Unknown country-code 'BB',Baylor Bolme,BB,1014,FR7619106000000001234567864,AGRIFRPP891,...,-0.018193,-0.977825,-0.024340,-1.049949,0.724633,-0.630278,2.892038,4.359151,7.188251,-3.455335
206753,901963,2023-02-03 23:49:54,356,FR7640618000000001234567882,BOUSFRPPXXX,Lezley Drew,FR,4020,FR7642559000000001234567890,CCOPFRPPXXX,...,0.112846,-1.556148,0.743775,-1.585202,0.743117,-0.603284,2.896064,4.353378,7.220034,-3.444787
206755,901965,2023-02-03 23:51:16,1769,FR7615930000000001234567870,NATXFRPPPAI,Bennet Bulengo,FR,3806,FR7624599000000001234567867,PRIVFRPPXXX,...,0.374923,-1.102453,0.848517,-1.202692,0.733875,-0.577744,2.900090,4.365910,7.204142,-3.421776


In [15]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)
#transactions_df.to_pickle('dataset.pkl')

train_df[input_features].info()

Mem. usage decreased to 10.63 Mb (46.7% reduction)
Mem. usage decreased to  2.83 Mb (46.7% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 68837 entries, 0 to 68836
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   TX_AMOUNT                            68837 non-null  float16
 1   TX_DURING_WEEKEND                    68837 non-null  float16
 2   TX_DURING_NIGHT                      68837 non-null  float16
 3   CUSTOMER_ID_NB_TX_1DAY_WINDOW        68837 non-null  float16
 4   CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW   68837 non-null  float16
 5   CUSTOMER_ID_NB_TX_7DAY_WINDOW        68837 non-null  float16
 6   CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW   68837 non-null  float16
 7   CUSTOMER_ID_NB_TX_30DAY_WINDOW       68837 non-null  float16
 8   CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW  68837 non-null  float16
 9   TERMINAL_ID_NB_TX_1DAY_WINDOW        68837 non-null  float

🛠️ Feature Engineering

In [16]:
#train_df.to_pickle('dataset.pkl')
train_df.to_pickle('dataset_sepa.pkl')

In [17]:
fraud_labels = transactions_df.copy()[["TRANSACTION_ID", "TX_FRAUD"]]
#fraud_labels.to_pickle('dataset_fraud.pkl')
fraud_labels.to_pickle('dataset_fraud_sepa.pkl')